* #### DJ Lee


In [1]:
import copy
import pandas as pd
import numpy as np
import datetime as dt
from dateutil.relativedelta import relativedelta
# from tqdm.notebook import tqdm
# import matplotlib.pyplot as plt
# import seaborn as sns

In [3]:
mdl_results_org = pd.read_excel("model_results_추가.xlsx",index_col=0)

In [4]:
mdl_results_org.tail()

,GDP(SA) True,DFM,BEM1,BEM2,BEM3,LBVAR,TFM,T-M,VAR,SVR-KNN,CRNN,RF-DVS,S-XGB,LQR,OptimARIMA
2023-06-01,0.6,1.6,0.79,0.79,0.70,NaN,NaN,NaN,0.88,0.53,NaN,0.62,1.12,0.23,0.70
2023-09-01,0.6,0.29,0.29,0.27,0.39,NaN,NaN,NaN,0.45,0.45,NaN,0.63,0.50,-0.66,0.08
2023-12-01,0.6,0.23,0.84,0.80,0.75,NaN,NaN,NaN,0.80,0.97,NaN,0.66,1.32,0.43,0.74
2024-03-01,1.3,0.69,0.70,0.70,0.66,NaN,NaN,NaN,0.62,0.82,NaN,0.71,0.94,0.83,0.95
NaT,NaN,.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# model_names = ['DFM',	'BEM1',	'BEM2',	'BEM3',	#'LBVAR',
            #    'TFM',	'T-M',	'VAR',	'SVR-KNN',	'CRNN',	'RF-DVS',	'S-XGB',	'LQR',	'OptimARIMA']
model_names = ['BEM1',	'BEM2', 'SVR-KNN','RF-DVS',	'S-XGB','LQR',	'OptimARIMA']
timeline = mdl_results_org.index

In [6]:
gdp_realized = mdl_results_org.iloc[0:-6, 0]
mdl_results = mdl_results_org.loc[timeline[0:-6], model_names].copy()

In [7]:
mdl_results.head()

,BEM1,BEM2,SVR-KNN,RF-DVS,S-XGB,LQR,OptimARIMA
2008-03-01,1.194740,1.245658,1.045777,1.556403,1.444494,1.196105,1.061182
2008-06-01,0.648690,0.695570,1.391040,0.820493,1.367215,1.411327,2.428746
2008-09-01,1.139184,0.932741,1.027312,1.025485,1.319489,0.204282,2.142860
2008-12-01,1.001532,1.401062,0.386686,0.705416,1.284721,-4.579140,-1.881478
2009-03-01,0.813676,3.515061,0.134462,0.364238,0.159652,-0.351205,0.460651


In [8]:
# 1. Mean, Median, Min, Max
mdl_results['Avg'] = mdl_results.loc[:,model_names].mean(axis=1)
mdl_results['Med'] = mdl_results.loc[:,model_names].median(axis=1)
mdl_results['Min'] = mdl_results.loc[:,model_names].min(axis=1)
mdl_results['Max'] = mdl_results.loc[:,model_names].max(axis=1)

In [9]:
def AdaMSM_4Q(mdl_results,gdp_realized):
    mdl_results_err = mdl_results.subtract(gdp_realized, axis=0)
    mdl_results_MAE4Q = mdl_results_err.abs().rolling(window=4).mean()
    mdl_results_MAE4Q['Top3Mdls'] = mdl_results_MAE4Q.apply(
        lambda row: tuple(row.nsmallest(3).index), axis=1 )
    mdl_results['AdaMSM'] = np.nan
    top3_lagged = mdl_results_MAE4Q['Top3Mdls'].shift(1)  
    for idx in mdl_results.index:
        if pd.notna(top3_lagged.loc[idx]):
            top3_cols = list(top3_lagged.loc[idx])
            # print(f"{idx}: {mdl_results.loc[idx, top3_cols].median()}")
            mdl_results.loc[idx,'AdaMSM'] = mdl_results.loc[idx, top3_cols].median() + mdl_results_MAE4Q.shift(1).loc[idx,mdl_results.columns[0]]*0 # Add 0 to remove first four results
    return mdl_results.loc[mdl_results.index,'AdaMSM']

In [10]:
mdl_results['AdaMSM'] = AdaMSM_4Q(mdl_results.loc[:,model_names],gdp_realized)

In [11]:
def dqs(preds: pd.DataFrame, lamb: float) -> pd.Series:
  taus = getTaus(preds, lamb)
  return pd.Series(map(lambda t, q: preds.loc[t].quantile(q), preds.index, taus), index=preds.index)

def getTaus(preds: pd.DataFrame, lamb: float) -> pd.Series:
  return ((np.sign(preds.mean(axis=1).diff().dropna()) * preds.std(axis=1).diff().dropna().apply(lambda x: max(0, x)) * preds.var(axis=1) * lamb).apply(lambda x: min(max(x, -0.5), 0.5)) + 0.5).fillna(0.5)

In [12]:
mdl_results['DQS'] = dqs(mdl_results.loc[:,model_names],2.0)

In [13]:
def dma(mdl_results: pd.DataFrame,gdp_realized,  alpha: float = 1,c: float = 0):
    mdl_results_err = mdl_results.subtract(gdp_realized, axis=0)
    mdl_results_MSE4Q = mdl_results_err.pow(2).rolling(window=4).mean()
    sigma2 = mdl_results_MSE4Q.shift(1).std(axis=1)**2
    likelihood = np.exp(-mdl_results_MSE4Q.shift(1).div(2 * sigma2, axis=0)).div(np.sqrt(2 * np.pi * sigma2),axis=0).pow(alpha) + c
    weights = likelihood.divide(likelihood.sum(axis=1),axis=0)
    return mdl_results.multiply(weights).sum(axis=1)


In [14]:
mdl_results['BMA'] = dma(mdl_results.loc[:,model_names],gdp_realized)
mdl_results['DMA'] = dma(mdl_results.loc[:,model_names],gdp_realized,.95,1e-3/14)

In [15]:
def lop(mdl_results: pd.DataFrame,gdp_realized: pd.core.series.Series):
    mdl_results_err = mdl_results.subtract(gdp_realized, axis=0)
    mdl_results_MSE4Q = mdl_results_err.pow(2).rolling(window=4).mean()
    weights=(1/mdl_results_MSE4Q.shift(1)).divide((1/mdl_results_MSE4Q.shift(1)).sum(axis=1),axis=0)
    return mdl_results.multiply(weights).sum(axis=1)


In [16]:
mdl_results['LOP'] = lop(mdl_results.loc[:,model_names],gdp_realized)

In [17]:
def AFTER(mdl_results: pd.DataFrame,gdp_realized: pd.core.series.Series):
    mdl_results_err = mdl_results.subtract(gdp_realized, axis=0)
    mdl_results_MSE4Q = mdl_results_err.pow(2).rolling(window=4).mean()
    weights=(1/np.exp(mdl_results_MSE4Q.shift(1))).divide((np.exp(-mdl_results_MSE4Q.shift(1))).sum(axis=1),axis=0)
    return mdl_results.multiply(weights).sum(axis=1)


In [18]:
mdl_results['AFTER'] = AFTER(mdl_results.loc[:,model_names],gdp_realized)

In [19]:
mdl_results.tail()

,BEM1,BEM2,SVR-KNN,RF-DVS,S-XGB,LQR,OptimARIMA,Avg,Med,Min,Max,AdaMSM,DQS,BMA,DMA,LOP,AFTER
2021-12-01,1.296681,1.352924,1.317400,0.746158,0.954301,1.735769,1.745013,1.306892,1.317400,0.746158,1.745013,1.352924,1.321771,1.486411,1.478066,1.345548,1.316971
2022-03-01,0.844207,0.836529,0.707758,0.719862,0.922648,0.893021,0.958533,0.840365,0.844207,0.707758,0.958533,0.836529,0.844207,0.872338,0.870864,0.849698,0.841999
2022-06-01,0.373763,0.391950,0.483481,0.185677,0.634784,0.268611,0.569141,0.415344,0.391950,0.185677,0.634784,0.373763,0.391597,0.369015,0.368277,0.381732,0.406384
2022-09-01,0.199928,0.095254,0.620047,0.102861,0.640976,0.588907,0.596038,0.406287,0.588907,0.095254,0.640976,0.199928,0.557983,0.157906,0.162712,0.334699,0.396748
2022-12-01,-0.784860,-0.932530,0.045649,0.154930,0.281753,-0.714974,-0.435121,-0.340736,-0.435121,-0.932530,0.281753,-0.784860,-0.631945,-0.861536,-0.810738,-0.467868,-0.352694


In [20]:
Ens_methods = ['DQS', 'Avg', 'Med', 'DMA', 'BMA', 'LOP', 'AFTER']

In [21]:
for col in ['AdaMSM','BMA','DMA','LOP','AFTER']:
    mdl_results.loc[mdl_results.index[0:4], col] = mdl_results.loc[mdl_results.index[0:4], 'Avg'] # 첫 4개 행 비워뒀던 Dynamic 조정하는 녀석들은 Mean으로 채움

In [22]:
mdl_results.head()

,BEM1,BEM2,SVR-KNN,RF-DVS,S-XGB,LQR,OptimARIMA,Avg,Med,Min,Max,AdaMSM,DQS,BMA,DMA,LOP,AFTER
2008-03-01,1.194740,1.245658,1.045777,1.556403,1.444494,1.196105,1.061182,1.249194,1.196105,1.045777,1.556403,1.249194,1.196105,1.249194,1.249194,1.249194,1.249194
2008-06-01,0.648690,0.695570,1.391040,0.820493,1.367215,1.411327,2.428746,1.251869,1.367215,0.648690,2.428746,1.251869,1.410940,1.251869,1.251869,1.251869,1.251869
2008-09-01,1.139184,0.932741,1.027312,1.025485,1.319489,0.204282,2.142860,1.113050,1.027312,0.204282,2.142860,1.113050,1.027312,1.113050,1.113050,1.113050,1.113050
2008-12-01,1.001532,1.401062,0.386686,0.705416,1.284721,-4.579140,-1.881478,-0.240172,0.705416,-4.579140,1.401062,-0.240172,-4.579140,-0.240172,-0.240172,-0.240172,-0.240172
2009-03-01,0.813676,3.515061,0.134462,0.364238,0.159652,-0.351205,0.460651,0.728076,0.364238,-0.351205,3.515061,0.134462,0.364238,0.568941,0.577116,0.214743,-0.146285


In [23]:
mdl_results.subtract(gdp_realized,axis=0).abs().mean()

BEM1          0.595239
BEM2          0.588807
SVR-KNN       0.465191
RF-DVS        0.539075
S-XGB         0.526805
LQR           0.539336
OptimARIMA    0.529347
Avg           0.424534
Med           0.451647
Min           0.534498
Max           0.850418
AdaMSM        0.460500
DQS           0.412152
BMA           0.465729
DMA           0.463376
LOP           0.434639
AFTER         0.424466
dtype: float64

Data-driven Lambda with new DQS(use IQR instead of standard deviation)

In [24]:
def dqs_Mk2(preds: pd.DataFrame,gdp_series: pd.Series) -> pd.Series:
  lambdas = get_lambdas(preds,gdp_series)
  taus = getTaus_Mk2(preds, lambdas)
  return pd.Series(map(lambda t, q: preds.loc[t].quantile(q), preds.index, taus), index=preds.index)

def getTaus_Mk2(preds: pd.DataFrame, lamb) -> pd.Series:
  iqr_ = preds.quantile(.75,axis=1)-preds.quantile(.25,axis=1)
  # iqr_ = preds.std(axis=1)
  return ((np.sign(preds.mean(axis=1).diff().dropna()) * (iqr_.diff().dropna().apply(lambda x: max(0, x))) * iqr_.pow(2) * lamb).apply(lambda x: min(max(x, -0.5), 0.5)) + 0.5).fillna(0.5)

def iqr(series):
    return series.quantile(1-.25)-series.quantile(.25)

def get_lambdas(preds, gdp_series):
    history_window = preds.index.values[np.newaxis, :] > gdp_series.index.values[:, np.newaxis]
    iqr_series = pd.Series(
    [gdp_series[history_window[:, i]].quantile(.75) - 
     gdp_series[history_window[:, i]].quantile(.25) 
     for i in range(history_window.shape[1])],
    index=preds.index)
    # iqr_series = pd.Series(
    # [gdp_series[history_window[:, i]].std()
    #  for i in range(history_window.shape[1])],
    # index=preds.index)
    return 1/iqr_series**2

In [25]:
mdl_results['DQS_Mk2'] = dqs_Mk2(mdl_results,gdp_realized)

In [26]:
mdl_results.subtract(gdp_realized,axis=0).abs().mean()

BEM1          0.595239
BEM2          0.588807
SVR-KNN       0.465191
RF-DVS        0.539075
S-XGB         0.526805
LQR           0.539336
OptimARIMA    0.529347
Avg           0.424534
Med           0.451647
Min           0.534498
Max           0.850418
AdaMSM        0.460500
DQS           0.412152
BMA           0.465729
DMA           0.463376
LOP           0.434639
AFTER         0.424466
DQS_Mk2       0.423786
dtype: float64

In [27]:
mdl_results.to_csv("ens_result_추가.csv",index=True)

In [28]:
from scipy import stats

In [29]:
def get_percentile(row, realized_series):
    actual_val = realized_series[row.name]   
    return stats.percentileofscore(row, actual_val, kind='rank')

Outliers deleted

In [30]:
def removeOutlier(preds: pd.DataFrame) -> pd.Series:
  preds = copy.deepcopy(preds)
  for t in preds.index:
    lqr = preds.loc[t].quantile(0.75) - preds.loc[t].quantile(0.25)
    lowerBound = preds.loc[t].quantile(0.25) - 1.5 * lqr
    upperBound = preds.loc[t].quantile(0.75) + 1.5 * lqr
    for col in preds.columns:
      if not lowerBound <= preds.loc[t, col] <= upperBound:
        preds.loc[t, col] = np.nan
  return preds


In [31]:
mdl_results_clean = removeOutlier(mdl_results.loc[:,model_names].copy())
mdl_results_clean.head()

,BEM1,BEM2,SVR-KNN,RF-DVS,S-XGB,LQR,OptimARIMA
2008-03-01,1.194740,1.245658,1.045777,1.556403,1.444494,1.196105,1.061182
2008-06-01,0.648690,0.695570,1.391040,0.820493,1.367215,1.411327,NaN
2008-09-01,1.139184,0.932741,1.027312,1.025485,1.319489,NaN,NaN
2008-12-01,1.001532,1.401062,0.386686,0.705416,1.284721,NaN,-1.881478
2009-03-01,0.813676,NaN,0.134462,0.364238,0.159652,-0.351205,0.460651


In [32]:
mdl_results_clean['Avg'] = mdl_results_clean.loc[:,model_names].mean(axis=1)
mdl_results_clean['Med'] = mdl_results_clean.loc[:,model_names].median(axis=1)
mdl_results_clean['Max'] = mdl_results_clean.loc[:,model_names].max(axis=1)
mdl_results_clean['Min'] = mdl_results_clean.loc[:,model_names].min(axis=1)
mdl_results_clean['AdaMSM'] = AdaMSM_4Q(mdl_results_clean.loc[:,model_names],gdp_realized)
mdl_results_clean['DQS'] = dqs(mdl_results_clean.loc[:,model_names],2.0)
mdl_results_clean['BMA'] = dma(mdl_results_clean.loc[:,model_names],gdp_realized)
mdl_results_clean['DMA'] = dma(mdl_results_clean.loc[:,model_names],gdp_realized,.95,1e-3/14)

In [33]:
for col in ['AdaMSM','BMA','DMA','LOP','AFTER']:
    mdl_results_clean.loc[mdl_results_clean.index[0:4], col] = mdl_results_clean.loc[mdl_results_clean.index[0:4], 'Avg'] # 첫 4개 행 비워뒀던 Dynamic 조정하는 녀석들은 Mean으로 채움

In [34]:
mdl_results_clean.subtract(gdp_realized,axis=0).abs().mean(skipna=False)

BEM1               NaN
BEM2               NaN
SVR-KNN       0.465191
RF-DVS             NaN
S-XGB              NaN
LQR                NaN
OptimARIMA         NaN
Avg           0.435808
Med           0.453874
Max           0.726504
Min           0.504423
AdaMSM             NaN
DQS           0.436318
BMA           0.491490
DMA           0.484318
LOP                NaN
AFTER              NaN
dtype: float64

In [35]:
mdl_results_clean.to_excel('ens.xlsx')